In [2]:
import pandas as pd 
import numpy as np
from pandas import ExcelWriter
import openpyxl
import geopandas as gpd
import itertools
pd.set_option('display.max_columns', None)

In [3]:
input_path='../../data/input/dhis2/new_system/'
output_path='../../data/output/sprint3_analysis/'
shapes_path='../../data/shapes/district/districts_17_19.shp'
input_path_old='../../data/input/dhis2/old_system/'

In [4]:
#Antental
df_Jan = pd.read_csv(input_path+'antenatal_jan.csv')
df_Feb = pd.read_csv(input_path+'antenatal_feb.csv')
df_Mar = pd.read_csv(input_path+'antenatal_mar.csv')
df_Apr = pd.read_csv(input_path+'antenatal_april.csv')

In [5]:
districts = df_Feb[df_Feb['Data']=='105-AN02. ANC 4th Visit for women']['Organisation unit']

In [6]:
def get_clean_pivot_from_stack(indicator):
    df = pd.DataFrame(districts).reset_index(drop=True)

    Jan = df_Jan[df_Jan['Data']==indicator].drop('Data', axis=1)
    Feb = df_Feb[df_Feb['Data']==indicator].drop('Data', axis=1)
    March = df_Mar[df_Mar['Data']==indicator].drop('Data', axis=1)
    April = df_Apr[df_Apr['Data']==indicator].drop('Data', axis=1)
    
    #print(Jan.shape[0]+ Feb.shape[0]+ March.shape[0] + April.shape[0])

    df_final = Jan.merge(Feb,on='Organisation unit', how='outer').merge(March,on='Organisation unit', how='outer').merge(April,on='Organisation unit', how='outer')
    #df_final.columns = ["Districts", "Jan_"+indicator, "Feb_"+indicator, "Mar_"+indicator, "Apr_"+indicator]
    df_final.set_index('Organisation unit',drop=True, inplace=True)
    columns = [('2020',indicator,"Jan"), ('2020',indicator,"Feb"),('2020',indicator,"Mar"),('2020',indicator,"Apr")]
    df_final.columns=pd.MultiIndex.from_tuples(columns)
    return df_final

In [7]:

# Creating a dict of names to replace dsirtcit names

def get_district_name_dict (df):
    keys = list(set(df.index).difference(set(shapes['name_19'])))
    values = list(set(shapes['name_19']).difference(set(df.index)))
    district_name = dict(zip(keys, values))
    return district_name

In [8]:

# build a small fucntion to split the string column name of the data download as pivot 

def split(strng, sep, occ):
    strng = strng.split(sep)
    return sep.join(strng[occ[1]:]), sep.join(strng[:occ[0]]), sep.join(strng[occ[0]:occ[1]])[:3]

In [9]:
# To clean the data downloaded in a pivot format

def get_clean_pivot(df):
    df['district']=df['organisationunitname'].apply(lambda x: x[:-9].upper())
    df.set_index('district',drop=True,inplace=True)
    #cols = np.arange(0,4)
    #df.drop(df.columns[cols],axis=1,inplace=True)
    df.drop('organisationunitname',axis=1,inplace=True)
    df.rename(index=get_district_name_dict(df),inplace=True)
    cols=df.columns
    new_cols=[]
    for col in cols:
        new_cols.append(split(col,' ',[-2,-1]))
    df.columns=pd.MultiIndex.from_tuples(new_cols)
    return df

In [10]:
def get_clean_pivot1(df):
    df['district']=df['organisationunitname'].str.upper()
    df.set_index('district',drop=True,inplace=True)
    #cols = np.arange(0,4)
    df.drop('organisationunitname',axis=1,inplace=True)
    df.rename(index=get_district_name_dict(df),inplace=True)
    cols=df.columns
    new_cols=[]
    for col in cols:
        new_cols.append(split(col,' ',[-2,-1]))
    df.columns=pd.MultiIndex.from_tuples(new_cols)
    return df

In [11]:
# To get year on year deltas over a list of months

def get_delta(df,st_year,end_year,indic,st_month,end_month):
    delta=((df[(end_year,indicator,end_month)]
            -df[(st_year,indicator,st_month)])
           /df[(st_year,indicator,st_month)])
    return delta

In [12]:
def get_yoy_delta(df,indic,period):
    tot_18 = tot_19= tot_20 = np.zeros(135)
    for month in period:
        tot_18= tot_18+list(df[('2018',indic,month)])
        tot_19= tot_19+list(df[('2019',indic,month)])
        tot_20= tot_20+list(df[('2020',indic,month)])
    delta =  (tot_20-tot_19)/tot_19
    return tot_18,tot_19,tot_20,delta

## ANC Data from new system 

In [13]:

multi_index=pd.MultiIndex.from_tuples([('Test','test','test')])
anc_new =pd.DataFrame(index=districts,columns=multi_index)

In [14]:
indicators = df_Jan['Data'].unique()
for indicator in indicators:
    df_months = get_clean_pivot_from_stack(indicator)
    anc_new = pd.merge(anc_new, df_months, how='left',left_index=True,right_index=True)

In [15]:
anc_new.shape

(135, 297)

In [16]:
anc_new.drop('test',axis=1,inplace=True,level=2)
null_sum1= anc_new.count()/135


## PNC and MAT Data from new system

In [17]:
#Postnatl and Maternity 

df_Jan = pd.read_csv(input_path+'Mat_Post_jan.csv')
df_Feb = pd.read_csv(input_path+'Mat_Post_feb.csv')
df_Mar = pd.read_csv(input_path+'Mat_Post_mar.csv')
df_Apr = pd.read_csv(input_path+'Mat_Post_apr.csv')

In [18]:
districts = df_Feb[df_Feb['Data']=='105-PN01a. Post Natal Attendances - Mother']['Organisation unit'].unique()
multi_index = pd.MultiIndex.from_tuples([('Test','test','test')])
pnc_mat_new = pd.DataFrame(index=districts,columns=multi_index)

In [19]:
indicators = df_Jan['Data'].unique()
for indicator in indicators:
    df_months = get_clean_pivot_from_stack(indicator)
    pnc_mat_new = pd.merge(pnc_mat_new, df_months, how='left',left_index=True,right_index=True)
pnc_mat_new.shape

(135, 333)

In [20]:
pnc_mat_new.drop('test',axis=1,inplace=True,level=2)
null_sum2= pnc_mat_new.count()/135

## combine pnc, anc and mat data and match district to shapefiles

In [21]:
null_sum=pd.concat([null_sum1,null_sum2])
null_sum_df = pd.DataFrame(null_sum)
null_sum_df.columns = ['% of data available']
null_sum_df.index.rename(['year','indicator','month'],inplace=True)
null_av_df=null_sum_df.groupby('indicator')['% of data available'].mean()

In [22]:
mnch_new_df = pd.merge(anc_new,pnc_mat_new,how='inner',left_index=True,right_index=True)
mnch_new_df.shape

(135, 628)

In [23]:
mnch_new_df.rename(index=lambda s: s [:-9],inplace=True)
mnch_new_df.rename(index=str.upper,inplace=True)
mnch_new_df.head(3)

2020  \
           105-AN25b. Pregnant women dewormed or receiving mebendazole - Done after 28 weeks   
                                                                                         Jan   
KAPCHORWA                                                53.0                                  
BUNYANGABU                                              142.0                                  
NAMAYINGO                                               250.0                                  

                               \
                                
            Feb    Mar    Apr   
KAPCHORWA    72  111.0  325.0   
BUNYANGABU  278  201.0  456.0   
NAMAYINGO   432  339.0  261.0   

                                                                                \
           105-AN18. Pregnant women with Hb <10g/dl at ANC 1st contact / visit   
                                                                           Jan   
KAPCHORWA                                                 2.0                    
BUNYANGABU                                                NaN                    
NAMAYINGO                                                 0.0                    

                           \
                            
            Feb  Mar  Apr   
KAPCHORWA   0.0  0.0  0.0   
BUNYANGABU  3.0  NaN  NaN   
NAMAYINGO   0.0  0.0  0.0   

                                                                          \
           105-AN12b. TB Screening for ANC clients - Presumed to have TB   
                                                                     Jan   
KAPCHORWA                                                 0.0              
BUNYANGABU                                               10.0              
NAMAYINGO                                                 5.0              

                              \
                               
             Feb   Mar   Apr   
KAPCHORWA    3.0   0.0   1.0   
BUNYANGABU  14.0  26.0  11.0   
NAMAYINGO    3.0   0.0  54.0   

                                                                              \
           105-AN15b. Blood and Rhesus Group (B) - Rhesus(B+)                  
                                                          Jan  Feb  Mar  Apr   
KAPCHORWA                                                 0.0  0.0  5.0  6.0   
BUNYANGABU                                                NaN  1.0  2.0  NaN   
NAMAYINGO                                                 0.0  0.0  0.0  4.0   

                                                                                               \
           105-AN26a. Pregnant Women tested for syphilis for first time during this pregnancy   
                                                                                          Jan   
KAPCHORWA                                               474.0                                   
BUNYANGABU                                              698.0                                   
NAMAYINGO                                               882.0                                   

                               \
                                
            Feb    Mar    Apr   
KAPCHORWA   348  383.0  297.0   
BUNYANGABU  565  576.0  517.0   
NAMAYINGO   856  535.0  575.0   

                                                                                              \
           105-AN37a. Pregnant women given self-testing kits for their male partners - Total   
                                                                                         Jan   
KAPCHORWA                                                 0.0                                  
BUNYANGABU                                               36.0                                  
NAMAYINGO                                                 1.0                                  

                              \
                               
             Feb   Mar   Apr   
KAPCHORWA    0.0   0.0   0.0   
BUNYANGABU  57.0  28.0  67

In [24]:
shapes = gpd.read_file(shapes_path)
mnch_new_df.rename(index=get_district_name_dict(mnch_new_df),inplace=True)


## Selecting both relevant and well reported indicators¶


In [25]:
target = pd.read_excel(input_path+'Data summary.xlsx',sheet_name='target')

In [26]:
target = pd.merge(target,null_av_df,left_on='Var',right_on='indicator',how='left')
target['keep']=(target['% of data available']>0.7)

In [27]:
mnch_target = [ 
#'105-AN010. Third dose IPT (IPT3)',
'105-AN01a. ANC 1st Visit for women',
'105-AN02. ANC 4th Visit for women',
#'105-AN05. Total ANC contacts/visits (New clients + Re-attendances)',
'105-MA01. Admissions',
'105-MA04a. Deliveries in unit - Total',
#'105-MA02b. Referrals to maternity unit - Community services',
'105-MA04b1. Deliveries in unit -Live births - Total',
#'105-PN02b. Referrals to PNC unit - Community services',
'105-PN01a. Post Natal Attendances - Mother',
#'105-PN01b. Post Natal Attendances - Baby',
#'105-PN14a. Lactating mothers who received Nutritional assessment - Total',
#'105-PN15a. Post natal mothers who received Counselling (Maternal Nutritional) - Total'
'105-MA04d1. Deliveries in unit - Macerated still birth - Total',
'105-MA04c1. Deliveries in unit - Fresh still birth - Total',
'105-MA11. Newborn deaths (0-7 days)',

]


In [28]:
mnch_new_df=mnch_new_df.iloc[:, mnch_new_df.columns.get_level_values(1).isin(mnch_target)].copy()
mnch_new_df.shape

(135, 36)

In [29]:
#for m in ['Jan','Feb','Mar','Apr']:
    #mnch_new_df[('2020','Deliveries in unit - still birth total',m)]=(mnch_new_df[('2020','105-MA04d1. Deliveries in unit - Macerated still birth - Total',m)]
                                                                      #+mnch_new_df[('2020','105-MA04c1. Deliveries in unit - Fresh still birth - Total',m)])
    
all_months=['Jan','Feb','Mar','Apr']

for m in all_months:
    mnch_new_df[('2020','Deliveries in unit - still birth total',m)]=(mnch_new_df[('2020','105-MA04d1. Deliveries in unit - Macerated still birth - Total',m)]
                                                                      +mnch_new_df[('2020','105-MA04c1. Deliveries in unit - Fresh still birth - Total',m)])
    mnch_new_df[('2020','Deliveries in unit - disaggregated birth total',m)]=(mnch_new_df[('2020','Deliveries in unit - still birth total',m)]
                                                                      +mnch_new_df[('2020','105-MA04b1. Deliveries in unit -Live births - Total',m)])
    mnch_new_df[('2020','Deliveries in unit - still birth discrepencies',m)]=(mnch_new_df[('2020','105-MA04a. Deliveries in unit - Total',m)]
                                                                      -mnch_new_df[('2020','Deliveries in unit - disaggregated birth total',m)])

In [30]:
mnch_new_df.head()

2020                        \
           105-AN01a. ANC 1st Visit for women                         
                                          Jan   Feb     Mar     Apr   
KAPCHORWA                               470.0   400   399.0   395.0   
BUNYANGABU                              848.0   601   673.0   785.0   
NAMAYINGO                              1076.0  1034   858.0  1009.0   
MBARARA                                1841.0  1599  1575.0   960.0   
IGANGA                                 1817.0  1635  1377.0  1322.0   

                                                                   \
           105-AN02. ANC 4th Visit for women                        
                                         Jan    Feb    Mar    Apr   
KAPCHORWA                              196.0    191  173.0  235.0   
BUNYANGABU                             374.0  27256  316.0  390.0   
NAMAYINGO                              222.0    285  276.0  303.0   
MBARARA                               1006.0    920  896.0  547.0   
IGANGA                                 572.0    718  554.0  654.0   

                                                                               \
           105-PN01a. Post Natal Attendances - Mother                           
                                                  Jan     Feb     Mar     Apr   
KAPCHORWA                                       655.0   441.0   352.0   429.0   
BUNYANGABU                                      841.0   856.0   996.0   811.0   
NAMAYINGO                                       585.0   799.0   812.0   641.0   
MBARARA                                        2013.0  1790.0  1991.0  1459.0   
IGANGA                                         1482.0  1631.0  1386.0  1360.0   

                                                                        \
           105-MA04b1. Deliveries in unit -Live births - Total           
                                                           Jan     Feb   
KAPCHORWA                                               385.0    389.0   
BUNYANGABU                                              332.0    335.0   
NAMAYINGO                                               290.0    296.0   
MBARARA                                                1583.0   1528.0   
IGANGA                                                  969.0   1069.0   

                                                                            \
                           105-MA11. Newborn deaths (0-7 days)               
               Mar     Apr                                 Jan   Feb   Mar   
KAPCHORWA    355.0   320.0                                 3.0   0.0   1.0   
BUNYANGABU   311.0   493.0                                 1.0   1.0   0.0   
NAMAYINGO    278.0   267.0                                 4.0   2.0   1.0   
MBARARA     1345.0  1082.0                                 4.0   2.0   3.0   
IGANGA      1080.0   911.0                                15.0  13.0  10.0   

                                                                        \
                 105-MA04a. Deliveries in unit - Total                   
             Apr                                   Jan     Feb     Mar   
KAPCHORWA    4.0                                 393.0   395.0   366.0   
BUNYANGABU   NaN                                 354.0   389.0   317.0   
NAMAYINGO    2.0                                 243.0   322.0   332.0   
MBARARA      4.0                                1678.0  1494.0  1390.0   
IGANGA      12.0                                 996.0  1076.0  1080.0   

                    \
                     
               Apr   
KAPCHORWA    324.0   
BUNYANGABU   495.0   
NAMAYINGO    287.0   
MBARARA     1128.0   
IGANGA       905.0   

                                                                           \
           105-MA04d1. Deliveries in unit - Macerated still birth - Total   
                                                                      Jan   
KAPCHORWA                                          

In [31]:
mnch_target.append('Deliveries in unit - still birth total')
mnch_target.append('Deliveries in unit - disaggregated birth total')
mnch_target.append('Deliveries in unit - still birth discrepencies')

In [32]:
mnch_target

['105-AN01a. ANC 1st Visit for women',
 '105-AN02. ANC 4th Visit for women',
 '105-MA01. Admissions',
 '105-MA04a. Deliveries in unit - Total',
 '105-MA04b1. Deliveries in unit -Live births - Total',
 '105-PN01a. Post Natal Attendances - Mother',
 '105-MA04d1. Deliveries in unit - Macerated still birth - Total',
 '105-MA04c1. Deliveries in unit - Fresh still birth - Total',
 '105-MA11. Newborn deaths (0-7 days)',
 'Deliveries in unit - still birth total',
 'Deliveries in unit - disaggregated birth total',
 'Deliveries in unit - still birth discrepencies']

In [33]:
print(null_sum.loc['2020',mnch_target,['Jan','Feb','Mar','Apr']])


year  indicator                                                       month
2020  105-AN01a. ANC 1st Visit for women                              Jan      1.000000
                                                                      Feb      1.000000
                                                                      Mar      1.000000
                                                                      Apr      1.000000
      105-AN02. ANC 4th Visit for women                               Jan      1.000000
                                                                      Feb      1.000000
                                                                      Mar      1.000000
                                                                      Apr      1.000000
      105-PN01a. Post Natal Attendances - Mother                      Jan      1.000000
                                                                      Feb      1.000000
                                            

In [34]:
for indicator in mnch_target:
    mnch_new_df[('2020',indicator,'delta_jan_20_apr_20')]=get_delta(df=mnch_new_df,
                                                        st_year='2020',
                                                        end_year='2020',
                                                        indic=indicator,
                                                        st_month='Jan',
                                                        end_month='Apr')

In [35]:
mnch_new_df.shape

(135, 60)

In [36]:
#105-AN02. ANC 4th Visit for women

# Add in the MNCH data from the old DHIS2 instance¶


In [37]:
old_anc = pd.read_csv(input_path_old+"opd_antenatal.csv")
old_mat = pd.read_csv(input_path_old+"opd_maternity.csv")
old_pnc = pd.read_csv(input_path_old+"opd_postnatal.csv")


In [38]:
all_months=['January','February','March','April','May','June','July','August','September','October','November','December']

mat_old_indics=['105-2.2 Newborn deaths (0-7days)','105-2.2b Deliveries in unit(Fresh Still births)','105-2.2c Deliveries in unit(Macerated still births)',
                '105-2.2d Deliveries in unit(Live Births)','105-2.2a Deliveries in unit']
years = ['2018','2019']

mat_death =[]

for a,c,b in itertools.product(mat_old_indics, years, all_months):
    mat_death.append(a + " " + b + ' ' + c)

In [39]:
pnc = ['organisationunitname',
'105-2.3 Postnatal Attendances January 2018',
'105-2.3 Postnatal Attendances February 2018',
'105-2.3 Postnatal Attendances March 2018',
'105-2.3 Postnatal Attendances April 2018',
'105-2.3 Postnatal Attendances May 2018',
'105-2.3 Postnatal Attendances June 2018',
'105-2.3 Postnatal Attendances July 2018',
'105-2.3 Postnatal Attendances August 2018',
'105-2.3 Postnatal Attendances September 2018',
'105-2.3 Postnatal Attendances October 2018',
'105-2.3 Postnatal Attendances November 2018',
'105-2.3 Postnatal Attendances December 2018',
'105-2.3 Postnatal Attendances January 2019',
'105-2.3 Postnatal Attendances February 2019',
'105-2.3 Postnatal Attendances March 2019',
'105-2.3 Postnatal Attendances April 2019',
'105-2.3 Postnatal Attendances May 2019',
'105-2.3 Postnatal Attendances June 2019',
'105-2.3 Postnatal Attendances July 2019',
'105-2.3 Postnatal Attendances August 2019',
'105-2.3 Postnatal Attendances September 2019',
'105-2.3 Postnatal Attendances October 2019',
'105-2.3 Postnatal Attendances November 2019',
'105-2.3 Postnatal Attendances December 2019'
]

anc = [
    'organisationunitname','105-2.1 A1:ANC 1st Visit for women January 2018',
'105-2.1 A1:ANC 1st Visit for women February 2018',
'105-2.1 A1:ANC 1st Visit for women March 2018',
'105-2.1 A1:ANC 1st Visit for women April 2018',
'105-2.1 A1:ANC 1st Visit for women May 2018',
'105-2.1 A1:ANC 1st Visit for women June 2018',
'105-2.1 A1:ANC 1st Visit for women July 2018',
'105-2.1 A1:ANC 1st Visit for women August 2018',
'105-2.1 A1:ANC 1st Visit for women September 2018',
'105-2.1 A1:ANC 1st Visit for women October 2018',
'105-2.1 A1:ANC 1st Visit for women November 2018',
'105-2.1 A1:ANC 1st Visit for women December 2018',
'105-2.1 A1:ANC 1st Visit for women January 2019',
'105-2.1 A1:ANC 1st Visit for women February 2019',
'105-2.1 A1:ANC 1st Visit for women March 2019',
'105-2.1 A1:ANC 1st Visit for women April 2019',
'105-2.1 A1:ANC 1st Visit for women May 2019',
'105-2.1 A1:ANC 1st Visit for women June 2019',
'105-2.1 A1:ANC 1st Visit for women July 2019',
'105-2.1 A1:ANC 1st Visit for women August 2019',
'105-2.1 A1:ANC 1st Visit for women September 2019',
'105-2.1 A1:ANC 1st Visit for women October 2019',
'105-2.1 A1:ANC 1st Visit for women November 2019',
'105-2.1 A1:ANC 1st Visit for women December 2019',
'105-2.1 A2:ANC 4th Visit for women January 2018',
'105-2.1 A2:ANC 4th Visit for women February 2018',
'105-2.1 A2:ANC 4th Visit for women March 2018',
'105-2.1 A2:ANC 4th Visit for women April 2018',
'105-2.1 A2:ANC 4th Visit for women May 2018',
'105-2.1 A2:ANC 4th Visit for women June 2018',
'105-2.1 A2:ANC 4th Visit for women July 2018',
'105-2.1 A2:ANC 4th Visit for women August 2018',
'105-2.1 A2:ANC 4th Visit for women September 2018',
'105-2.1 A2:ANC 4th Visit for women October 2018',
'105-2.1 A2:ANC 4th Visit for women November 2018',
'105-2.1 A2:ANC 4th Visit for women December 2018',
'105-2.1 A2:ANC 4th Visit for women January 2019',
'105-2.1 A2:ANC 4th Visit for women February 2019',
'105-2.1 A2:ANC 4th Visit for women March 2019',
'105-2.1 A2:ANC 4th Visit for women April 2019',
'105-2.1 A2:ANC 4th Visit for women May 2019',
'105-2.1 A2:ANC 4th Visit for women June 2019',
'105-2.1 A2:ANC 4th Visit for women July 2019',
'105-2.1 A2:ANC 4th Visit for women August 2019',
'105-2.1 A2:ANC 4th Visit for women September 2019',
'105-2.1 A2:ANC 4th Visit for women October 2019',
'105-2.1 A2:ANC 4th Visit for women November 2019',
'105-2.1 A2:ANC 4th Visit for women December 2019']

mat = [
'organisationunitname',
'105-2.2 Admissions January 2018',
'105-2.2 Admissions February 2018',
'105-2.2 Admissions March 2018',
'105-2.2 Admissions April 2018',
'105-2.2 Admissions May 2018',
'105-2.2 Admissions June 2018',
'105-2.2 Admissions July 2018',
'105-2.2 Admissions August 2018',
'105-2.2 Admissions September 2018',
'105-2.2 Admissions October 2018',
'105-2.2 Admissions November 2018',
'105-2.2 Admissions December 2018',
'105-2.2 Admissions January 2019',
'105-2.2 Admissions February 2019',
'105-2.2 Admissions March 2019',
'105-2.2 Admissions April 2019',
'105-2.2 Admissions May 2019',
'105-2.2 Admissions June 2019',
'105-2.2 Admissions July 2019',
'105-2.2 Admissions August 2019',
'105-2.2 Admissions September 2019',
'105-2.2 Admissions October 2019',
'105-2.2 Admissions November 2019',
'105-2.2 Admissions December 2019']

In [40]:
old_anc = old_anc[anc]
old_pnc =  old_pnc[pnc]
old_mat = old_mat[mat+mat_death]
old_mat.head()

,organisationunitname,105-2.2 Admissions January 2018,105-2.2 Admissions February 2018,105-2.2 Admissions March 2018,105-2.2 Admissions April 2018,105-2.2 Admissions May 2018,105-2.2 Admissions June 2018,105-2.2 Admissions July 2018,105-2.2 Admissions August 2018,105-2.2 Admissions September 2018,105-2.2 Admissions October 2018,105-2.2 Admissions November 2018,105-2.2 Admissions December 2018,105-2.2 Admissions January 2019,105-2.2 Admissions February 2019,105-2.2 Admissions March 2019,105-2.2 Admissions April 2019,105-2.2 Admissions May 2019,105-2.2 Admissions June 2019,105-2.2 Admissions July 2019,105-2.2 Admissions August 2019,105-2.2 Admissions September 2019,105-2.2 Admissions October 2019,105-2.2 Admissions November 2019,105-2.2 Admissions December 2019,105-2.2 Newborn deaths (0-7days) January 2018,105-2.2 Newborn deaths (0-7days) February 2018,105-2.2 Newborn deaths (0-7days) March 2018,105-2.2 Newborn deaths (0-7days) April 2018,105-2.2 Newborn deaths (0-7days) May 2018,105-2.2 Newborn deaths (0-7days) June 2018,105-2.2 Newborn deaths (0-7days) July 2018,105-2.2 Newborn deaths (0-7days) August 2018,105-2.2 Newborn deaths (0-7days) September 2018,105-2.2 Newborn deaths (0-7days) October 2018,105-2.2 Newborn deaths (0-7days) November 2018,105-2.2 Newborn deaths (0-7days) December 2018,105-2.2 Newborn deaths (0-7days) January 2019,105-2.2 Newborn deaths (0-7days) February 2019,105-2.2 Newborn deaths (0-7days) March 2019,105-2.2 Newborn deaths (0-7days) April 2019,105-2.2 Newborn deaths (0-7days) May 2019,105-2.2 Newborn deaths (0-7days) June 2019,105-2.2 Newborn deaths (0-7days) July 2019,105-2.2 Newborn deaths (0-7days) August 2019,105-2.2 Newborn deaths (0-7days) September 2019,105-2.2 Newborn deaths (0-7days) October 2019,105-2.2 Newborn deaths (0-7days) November 2019,105-2.2 Newborn deaths (0-7days) December 2019,105-2.2b Deliveries in unit(Fresh Still births) January 2018,105-2.2b Deliveries in unit(Fresh Still births) February 2018,105-2.2b Deliveries in unit(Fresh Still births) March 2018,105-2.2b Deliveries in unit(Fresh Still births) April 2018,105-2.2b Deliveries in unit(Fresh Still births) May 2018,105-2.2b Deliveries in unit(Fresh Still births) June 2018,105-2.2b Deliveries in unit(Fresh Still births) July 2018,105-2.2b Deliveries in unit(Fresh Still births) August 2018,105-2.2b Deliveries in unit(Fresh Still births) September 2018,105-2.2b Deliveries in unit(Fresh Still births) October 2018,105-2.2b Deliveries in unit(Fresh Still births) November 2018,105-2.2b Deliveries in unit(Fresh Still births) December 2018,105-2.2b Deliveries in unit(Fresh Still births) January 2019,105-2.2b Deliveries in unit(Fresh Still births) February 2019,105-2.2b Deliveries in unit(Fresh Still births) March 2019,105-2.2b Deliveries in unit(Fresh Still births) April 2019,105-2.2b Deliveries in unit(Fresh Still births) May 2019,105-2.2b Deliveries in unit(Fresh Still births) June 2019,105-2.2b Deliveries in unit(Fresh Still births) July 2019,105-2.2b Deliveries in unit(Fresh Still births) August 2019,105-2.2b Deliveries in unit(Fresh Still births) September 2019,105-2.2b Deliveries in unit(Fresh Still births) October 2019,105-2.2b Deliveries in unit(Fresh Still births) November 2019,105-2.2b Deliveries in unit(Fresh Still births) December 2019,105-2.2c Deliveries in unit(Macerated still births) January 2018,105-2.2c Deliveries in unit(Macerated still births) February 2018,105-2.2c Deliveries in unit(Macerated still births) March 2018,105-2.2c Deliveries in unit(Macerated still births) April 2018,105-2.2c Deliveries in unit(Macerated still births) May 2018,105-2.2c Deliveries in unit(Macerated still births) June 2018,105-2.2c Deliveries in unit(Macerated still births) July 2018,105-2.2c Deliveries in unit(Macerated still births) August 2018,105-2.2c Deliveries in unit(Macerated still births) September 2018,105-2.2c Deliveries in unit(Macerated still births) October 2018,105-2.2c Deliveries in unit(Macerated still births) November 201

### Rename the the columns to match the columns names in the new instance then merge pnc, anc and mat 

In [41]:
old_mat=get_clean_pivot(old_mat)
old_mat.head()

2018                                                  \
         105-2.2 Admissions                                                   
                        Jan   Feb   Mar   Apr   May   Jun   Jul   Aug   Sep   
district                                                                      
ABIM                    369   328   357   412   462   555   543   558   528   
ADJUMANI               1075  1147  1132  1174  1242  1196  1304  1425  1282   
AGAGO                   993   935   963   801  1043   938  1040  1242  1039   
ALEBTONG                616   495   472   518   564   527   554   714   652   
AMOLATAR                374   376   362   381   345   306   350   392   366   

                                         2019                                \
                           105-2.2 Admissions                                 
           Oct   Nov   Dec                Jan   Feb   Mar   Apr   May   Jun   
district                                                                      
ABIM       529   522   565                427   412   398   401   429   471   
ADJUMANI  1400  1445  1333               1313  1292  2060  1218  1398  1319   
AGAGO     1016   934   935               1170   976  1029   892  1018  1048   
ALEBTONG   690   656   661                631   559   585   530   626   642   
AMOLATAR   397   372   419                376   294   372   336   355   359   

                                                                         2018  \
                                             105-2.2 Newborn deaths (0-7days)   
           Jul   Aug   Sep   Oct   Nov   Dec                              Jan   
district                                                                        
ABIM       504   431   484   343   307   285                              1.0   
ADJUMANI  1325  1393  1498  1535  1429  1334                              9.0   
AGAGO     1070  1029   936   959   849   760                              1.0   
ALEBTONG   641   620   650   645   575   667                              3.0   
AMOLATAR   341   401   417   409   438   434                              3.0   

                                                                 \
                                                                  
          Feb  Mar  Apr  May  Jun  Jul  Aug  Sep  Oct  Nov  Dec   
district                                                          
ABIM      2.0  0.0  1.0  2.0  1.0  1.0  4.0  2.0  NaN  2.0  1.0   
ADJUMANI  3.0  4.0  3.0  9.0  4.0  8.0  5.0  6.0  3.0  8.0  3.0   
AGAGO     0.0  3.0  0.0  1.0  2.0  1.0  1.0  2.0  7.0  3.0  7.0   
ALEBTONG  4.0  4.0  6.0  3.0  7.0  1.0  1.0  2.0  1.0  5.0  5.0   
AMOLATAR  1.0  0.0  NaN  0.0  NaN  NaN  1.0  NaN  3.0  2.0  0.0   

                                     2019                                  \
         105-2.2 Newborn deaths (0-7days)                                   
                                      Jan  Feb  Mar   Apr  May  Jun   Jul   
district                                                                    
ABIM                                  0.0  1.0  2.0   1.0  1.0  2.0   1.0   
ADJUMANI                              7.0  4.0  4.0   3.0  6.0  9.0  11.0   
AGAGO                                12.0  9.0  6.0  16.0  9.0  8.0  13.0   
ALEBTONG                              1.0  2.0  3.0   NaN  1.0  2.0   2.0   
AMOLATAR                              3.0  NaN  3.0   NaN  1.0  NaN   1.0   

                                     \
                                      
           Aug  Sep   Oct  Nov  Dec   
district                              
ABIM       0.0  0.0   0.0  2.0  3.0   
ADJUMANI   7.0  2.0  10.0  2.0  2.0   
AGAGO     10.0  2.0   9.0  1.0  6.0   
ALEBTONG   2.0  5.0   1.0  3.0  0.0   
AMOLATAR   NaN  NaN   4.0  5.0  NaN   

                                                    2018                      \
         105-2.2b Deliveries in unit(Fresh Still births)                       
                                                     Jan  F

In [42]:

#dropped_months=['Jun','Jul','Aug','Sep','Oct','Nov','Dec']
#old_mat.drop(dropped_months,axis=1,inplace=True,level=2)
old_names= list(old_mat.columns.levels[1])

new_names=['105-MA01. Admissions',
           '105-MA11. Newborn deaths (0-7 days)',
           '105-MA04a. Deliveries in unit - Total',
           '105-MA04c1. Deliveries in unit - Fresh still birth - Total',
           '105-MA04d1. Deliveries in unit - Macerated still birth - Total',
           '105-MA04b1. Deliveries in unit -Live births - Total']
             

old_names= list(old_mat.columns.levels[1])

indicator_name = dict(zip(old_names, new_names))
old_mat.columns.set_levels(list(indicator_name.values()),level=1,inplace=True)

indicator_name

{'105-2.2 Admissions': '105-MA01. Admissions',
 '105-2.2 Newborn deaths (0-7days)': '105-MA11. Newborn deaths (0-7 days)',
 '105-2.2a Deliveries in unit': '105-MA04a. Deliveries in unit - Total',
 '105-2.2b Deliveries in unit(Fresh Still births)': '105-MA04c1. Deliveries in unit - Fresh still birth - Total',
 '105-2.2c Deliveries in unit(Macerated still births)': '105-MA04d1. Deliveries in unit - Macerated still birth - Total',
 '105-2.2d Deliveries in unit(Live Births)': '105-MA04b1. Deliveries in unit -Live births - Total'}

In [43]:
all_months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

for y,m in itertools.product(years,all_months):
    old_mat[(y,'Deliveries in unit - still birth total',m)]=(old_mat[(y,'105-MA04d1. Deliveries in unit - Macerated still birth - Total',m)]
                                                                      +old_mat[(y,'105-MA04c1. Deliveries in unit - Fresh still birth - Total',m)])
    old_mat[(y,'Deliveries in unit - disaggregated birth total',m)]=(old_mat[(y,'Deliveries in unit - still birth total',m)]
                                                                      +old_mat[(y,'105-MA04b1. Deliveries in unit -Live births - Total',m)])
    old_mat[(y,'Deliveries in unit - still birth discrepencies',m)]=(old_mat[(y,'105-MA04a. Deliveries in unit - Total',m)]
                                                                      -old_mat[(y,'Deliveries in unit - disaggregated birth total',m)])

In [44]:
old_mat.head()


2018                                                  \
         105-MA01. Admissions                                                   
                          Jan   Feb   Mar   Apr   May   Jun   Jul   Aug   Sep   
district                                                                        
ABIM                      369   328   357   412   462   555   543   558   528   
ADJUMANI                 1075  1147  1132  1174  1242  1196  1304  1425  1282   
AGAGO                     993   935   963   801  1043   938  1040  1242  1039   
ALEBTONG                  616   495   472   518   564   527   554   714   652   
AMOLATAR                  374   376   362   381   345   306   350   392   366   

                                           2019                                \
                           105-MA01. Admissions                                 
           Oct   Nov   Dec                  Jan   Feb   Mar   Apr   May   Jun   
district                                                                        
ABIM       529   522   565                  427   412   398   401   429   471   
ADJUMANI  1400  1445  1333                 1313  1292  2060  1218  1398  1319   
AGAGO     1016   934   935                 1170   976  1029   892  1018  1048   
ALEBTONG   690   656   661                  631   559   585   530   626   642   
AMOLATAR   397   372   419                  376   294   372   336   355   359   

                                              \
                                               
           Jul   Aug   Sep   Oct   Nov   Dec   
district                                       
ABIM       504   431   484   343   307   285   
ADJUMANI  1325  1393  1498  1535  1429  1334   
AGAGO     1070  1029   936   959   849   760   
ALEBTONG   641   620   650   645   575   667   
AMOLATAR   341   401   417   409   438   434   

                                        2018                                \
         105-MA11. Newborn deaths (0-7 days)                                 
                                         Jan  Feb  Mar  Apr  May  Jun  Jul   
district                                                                     
ABIM                                     1.0  2.0  0.0  1.0  2.0  1.0  1.0   
ADJUMANI                                 9.0  3.0  4.0  3.0  9.0  4.0  8.0   
AGAGO                                    1.0  0.0  3.0  0.0  1.0  2.0  1.0   
ALEBTONG                                 3.0  4.0  4.0  6.0  3.0  7.0  1.0   
AMOLATAR                                 3.0  1.0  0.0  NaN  0.0  NaN  NaN   

                                                                 2019       \
                                  105-MA11. Newborn deaths (0-7 days)        
          Aug  Sep  Oct  Nov  Dec                                 Jan  Feb   
district                                                                     
ABIM      4.0  2.0  NaN  2.0  1.0                                 0.0  1.0   
ADJUMANI  5.0  6.0  3.0  8.0  3.0                                 7.0  4.0   
AGAGO     1.0  2.0  7.0  3.0  7.0                                12.0  9.0   
ALEBTONG  1.0  2.0  1.0  5.0  5.0                                 1.0  2.0   
AMOLATAR  1.0  NaN  3.0  2.0  0.0                                 3.0  NaN   

                                                                \
                                                                 
          Mar   Apr  May  Jun   Jul   Aug  Sep   Oct  Nov  Dec   
district                                                         
ABIM      2.0   1.0  1.0  2.0   1.0   0.0  0.0   0.0  2.0  3.0   
ADJUMANI  4.0   3.0  6.0  9.0  11.0   7.0  2.0  10.0  2.0  2.0   
AGAGO     6.0  16.0  9.0  8.0  13.0  10.0  2.0   9.0  1.0  6.0   
ALEBTONG  3.0   NaN  1.0  2.0   2.0   2.0  5.0   1.0  3.0  0.0   
AMOLATAR  3.0   NaN  1.0  NaN   1.0   NaN  NaN   4.0  5.0  NaN   

                                                               2018            \
         105-MA04c1. Deliveries in unit - Fresh st

In [45]:
old_anc=get_clean_pivot(old_anc)


In [46]:
old_names= list(old_anc.columns.levels[1])

old_names

['105-2.1 A1:ANC 1st Visit for women', '105-2.1 A2:ANC 4th Visit for women']

In [47]:
new_names=['105-AN01a. ANC 1st Visit for women','105-AN02. ANC 4th Visit for women']
           
old_names= list(old_anc.columns.levels[1])

indicator_name = dict(zip(old_names, new_names))
old_anc.columns.set_levels(list(indicator_name.values()),level=1,inplace=True)

indicator_name

{'105-2.1 A1:ANC 1st Visit for women': '105-AN01a. ANC 1st Visit for women',
 '105-2.1 A2:ANC 4th Visit for women': '105-AN02. ANC 4th Visit for women'}

In [48]:
old_pnc=get_clean_pivot(old_pnc)

#dropped_months=['Jun','Jul','Aug','Sep','Oct','Nov','Dec']
#old_pnc.drop(dropped_months,axis=1,inplace=True,level=2)


new_names=['105-PN01a. Post Natal Attendances - Mother']
           
old_names= list(old_pnc.columns.levels[1])

indicator_name = dict(zip(old_names, new_names))
old_pnc.columns.set_levels(list(indicator_name.values()),level=1,inplace=True)

indicator_name

{'105-2.3 Postnatal Attendances': '105-PN01a. Post Natal Attendances - Mother'}

In [49]:
data=pd.merge(old_anc,old_mat,how='inner',left_index=True,right_index=True)

In [50]:
data = pd.merge(data,old_pnc,how='inner',left_index=True,right_index=True)

### Compute 2019 Deltas

In [51]:

# get the Jan-Apr deltas for 2019
for indicator in mnch_target:
    data[('2019',indicator,'delta_jan_19_apr_19')]=get_delta(df=data,
                                                        st_year='2019',
                                                        end_year='2019',
                                                        indic=indicator,
                                                        st_month='Jan',
                                                        end_month='Apr')

In [52]:
data.shape

(135, 300)

### Merge the old and the new mnch data and compute 19-20 Deltas

In [69]:
mnch_df=pd.merge(mnch_new_df,data,how='inner',left_index=True,right_index=True)


In [70]:
# Comparing official total and addition of dissagrates 

# Shows sum of aggregate is on average larger than offocial totals, andthat difference is hectic 
# Overall, the order of magintude of this is about 2% of total birth but sometimes very close to teh number of stillbirth
# Hard to draw a conclusion from that 

idx = pd.IndexSlice
all_birth = mnch_df.loc[:,idx[['2018','2019'],'105-MA04a. Deliveries in unit - Total',all_months]].sum()
discrepencies = mnch_df.loc[:,idx[['2018','2019'],'Deliveries in unit - still birth discrepencies',all_months]].sum()
still_birth = mnch_df.loc[:,idx[['2018','2019'],'Deliveries in unit - still birth total',all_months]].sum()

discrepencies

2018  Deliveries in unit - still birth discrepencies  Jan   -1753.0
                                                      Feb   -1352.0
                                                      Mar     -86.0
                                                      Apr    -155.0
                                                      May     309.0
                                                      Jun     161.0
                                                      Jul   -2625.0
                                                      Aug    -203.0
                                                      Sep   -1814.0
                                                      Oct    -258.0
                                                      Nov    -297.0
                                                      Dec    -316.0
2019  Deliveries in unit - still birth discrepencies  Jan      31.0
                                                      Feb    -358.0
                                                

In [71]:
# Outlier

mnch_df.loc['BUNYANGABU',('2020','105-AN02. ANC 4th Visit for women','Feb')]=np.nan

In [72]:
target = ['105-AN01a. ANC 1st Visit for women', 
          '105-AN02. ANC 4th Visit for women', 
          '105-MA01. Admissions', 
          '105-MA04c1. Deliveries in unit - Fresh still birth - Total', 
          '105-MA04d1. Deliveries in unit - Macerated still birth - Total',
          'Deliveries in unit - still birth total', 
          '105-MA11. Newborn deaths (0-7 days)', 
          '105-PN01a. Post Natal Attendances - Mother',
          'Deliveries in unit - still birth total',
          'Deliveries in unit - disaggregated birth total',
          'Deliveries in unit - still birth discrepencies',
          '105-MA04a. Deliveries in unit - Total',
          '105-MA04b1. Deliveries in unit -Live births - Total']

In [73]:
period=['Mar','Apr']

for indicator in target:
    (mnch_df[('2018',indicator,'period_total')],
    mnch_df[('2019',indicator,'period_total')],
    mnch_df[('2020',indicator,'period_total')],
    mnch_df[('2020',indicator,'delta_per_19_per_20')]) = get_yoy_delta(mnch_df,indicator,period)

<ipython-input-12-e9f19f73a366>:7: RuntimeWarning: divide by zero encountered in true_divide
  delta =  (tot_20-tot_19)/tot_19
<ipython-input-12-e9f19f73a366>:7: RuntimeWarning: invalid value encountered in true_divide
  delta =  (tot_20-tot_19)/tot_19


In [74]:
mnch_export_df=mnch_df.copy()
mnch_export_df.columns=mnch_df.columns.map("_".join)
mnch_export_df.to_csv(output_path+'mnch_map.csv')

In [75]:
indics_to_keep=list(mnch_df.columns.levels[1])
all_months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

mnch_stacked_df=pd.melt(mnch_df.copy().reset_index(), id_vars='index')

mnch_stacked_df.rename({'variable_0':'year',
                       'variable_1':'indicator',
                       'variable_2':'month',
                       'index':'district'},axis=1,inplace=True)


mnch_stacked_df= mnch_stacked_df[mnch_stacked_df.month.isin(all_months)]
mnch_stacked_df=mnch_stacked_df[mnch_stacked_df.indicator.isin(indics_to_keep)]

mnch_stacked_df.to_csv(output_path+'mnch_map_stacked.csv')

mnch_stacked_df.head()

,district,year,indicator,month,value
0,KAPCHORWA,2020,105-AN01a. ANC 1st Visit for women,Jan,470.0
1,BUNYANGABU,2020,105-AN01a. ANC 1st Visit for women,Jan,848.0
2,NAMAYINGO,2020,105-AN01a. ANC 1st Visit for women,Jan,1076.0
3,MBARARA,2020,105-AN01a. ANC 1st Visit for women,Jan,1841.0
4,IGANGA,2020,105-AN01a. ANC 1st Visit for women,Jan,1817.0


In [181]:
mnch_stacked_df[mnch_stacked_df['indicator']=='105-MA01. Admissions'].month.unique()

array(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
       'Oct', 'Nov', 'Dec'], dtype=object)